In [2]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "common/0_notebooks_base_setup.py"

default checking
Running command `conda list`... ok
jupyterlab=1.2.6 already installed
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
psutil=5.7.0 already installed
scipy=1.4.1 already installed
statsmodels=0.11.0 already installed
scikit-learn=0.22.1 already installed
xlrd=1.2.0 already installed
plotly=4.6.0 already installed
chart-studio=1.1.0 already installed
plotly-orca=1.3.1 already installed
Running command `conda install --yes unicodedata2`... ok
Solving environment: ...working... done

# All requested packages already installed.


Importando librerías ...
Las librerías has sido importadas con éxito


In [3]:
data_clean = pd.read_csv("CleanCSV/Clean02_DummiesYSuperficie.csv")

<hr id="PreciosYSuperficie">
<h2 style="color: lightblue">PRECIOS Y SUPERFICIE</h2>

<span style="font-size:18px">COMPLETAR price_aprox_usd<span>

In [5]:
regex = re.compile("USD\s+[\d|.]{3,10}", flags = re.IGNORECASE)
regexPrecio = data_clean[(data_clean.price_aprox_usd.isnull())].description.apply(lambda x: regex.search(str(x)))
resultado = regexPrecio.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace("usd", "").replace(".", "").strip()).astype(np.float64)
print("Antes: ", data_clean.price_aprox_usd.isnull().sum())
data_clean.loc[resultado.index, "price_aprox_usd"]  = resultado
print("Después: ", data_clean.price_aprox_usd.isnull().sum())

Antes:  20410
Después:  20274


In [6]:
regexPrecio = data_clean[(data_clean.price_aprox_usd.isnull())].title.apply(lambda x: regex.search(str(x)))
resultado = regexPrecio.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace("usd", "").replace(".", "").strip()).astype(np.float64)
print("Antes: ", data_clean.price_aprox_usd.isnull().sum())
data_clean.loc[resultado.index, "price_aprox_usd"]  = resultado
print("Después: ", data_clean.price_aprox_usd.isnull().sum())

Antes:  20274
Después:  20273


<span style="font-size:18px">COMPLETAR price_usd_per_m2<span>

<span style="font-size:16px; text-decoration: underline">NULOS price_usd_per_m2<span>

In [7]:
#Buscar nulos
priceUSDPerM2NullMask = data_clean.price_usd_per_m2.isnull()
print("Nulos: ", priceUSDPerM2NullMask.sum(), "\n")
display(data_clean[priceUSDPerM2NullMask].head(2))

Nulos:  52603 



,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,...,patio_dummy,jardin_dummy,quincho_dummy,aire acondicionado_dummy,amenities_dummy,estrenar_dummy,gimnasio_dummy,frente_dummy,subte_dummy,baulera_dummy
1,1,1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,...,False,False,False,False,False,True,False,True,False,False
3,3,3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,...,True,False,False,False,False,False,False,False,False,False


In [8]:
#Nulos por tipo de propiedad
popertyTypeGroupBy1 = data_clean[priceUSDPerM2NullMask].groupby(by="property_type")
popertyTypeGroupBy1["property_type"].count()

#Con esto se puede iterar sobre cada grupo y ver los registros por separado
#for name, group in popertyTypeGroupBy1: 
#    display(name) 
#    display(group) 
#    print() 

property_type
Fondo de Comercio       93
PH                    2497
apartment            28178
house                19909
store                 1926
Name: property_type, dtype: int64

<span style="font-size:16px; text-decoration: underline">NULOS price_per_m2<span>

In [9]:
#Buscar nulos
pricePerM2NullMask = data_clean.price_per_m2.isnull()
print(pricePerM2NullMask.sum())
display(data_clean[pricePerM2NullMask].filter(items=["price_per_m2", "currency"]).head(5))

33562


,price_per_m2,currency
1,NaN,USD
3,NaN,USD
5,NaN,NaN
9,NaN,ARS
10,NaN,USD


In [10]:
#Nulos por tipo de propiedad
popertyTypeGroupBy2 = data_clean[pricePerM2NullMask].groupby(by="property_type")
popertyTypeGroupBy2["property_type"].count()

property_type
Fondo de Comercio       85
PH                    1199
apartment            18871
house                11915
store                 1492
Name: property_type, dtype: int64

<span style="font-size:16px; text-decoration: underline">IMPUTAR VALORES EN price_usd_per_m2 a partir de price_per_m2<span>

In [11]:
#Buscar price_per_m2 en USD para imputar en price_usd_per_m2
pricePerM2NoNulos = data_clean[(priceUSDPerM2NullMask) & \
                   (~pricePerM2NullMask) & \
                   (data_clean.currency == "USD")].price_per_m2
display(pricePerM2NoNulos.head(3))

68    3263.888889
77    1764.705882
79    2809.523810
Name: price_per_m2, dtype: float64

In [12]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
data_clean.loc[pricePerM2NoNulos.index, "price_usd_per_m2"] = pricePerM2NoNulos
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())


Nulos antes:  52603
Nulos después:  31166


In [13]:
#Buscar price_por_m2 en pesos
display(data_clean[(priceUSDPerM2NullMask) & \
                   (~pricePerM2NullMask) & \
                   (data_clean.currency == "ARG")].filter(items=["price_usd_per_m2", "price_per_m2", "currency"]))

,price_usd_per_m2,price_per_m2,currency


<span style="font-size:16px; text-decoration: underline">IMPUTAR VALORES EN price_usd_per_m2 CRUZANDO POR UBICACIONES y precio_aprox_usd<span>

In [14]:
#Apartment sin price_usd_per_m2 con price_aprox_usd
data_clean["Indice"] = data_clean.index
dfGrupo1 = data_clean[(data_clean.price_usd_per_m2.isnull()) & (data_clean.price_aprox_usd.notnull())].filter(items=["property_type", "Indice", "state_name", "place_name", "place_with_parent_names", "price_aprox_usd"])
#Apartment con price_usd_per_m2 y con price_aprox_usd
dfGrupo2 = data_clean[(data_clean.price_usd_per_m2.notnull()) & (data_clean.price_aprox_usd.notnull())].filter(items=["property_type", "state_name", "place_name", "place_with_parent_names", "price_usd_per_m2", "price_aprox_usd"])

In [15]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "place_with_parent_names", "price_aprox_usd"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())


Nulos antes:  31166
Nulos después:  25115


In [16]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "place_name", "price_aprox_usd"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())

Nulos antes:  25115
Nulos después:  24826


In [17]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "state_name", "price_aprox_usd"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())

Nulos antes:  24826
Nulos después:  22839


In [18]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "place_with_parent_names"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())

Nulos antes:  22839
Nulos después:  20389


In [19]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "place_name"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())

Nulos antes:  20389
Nulos después:  20365


In [20]:
print("Nulos antes: ", data_clean.price_usd_per_m2.isnull().sum())
gruposMerge = pd.merge(dfGrupo1, dfGrupo2, how="inner", on=["property_type", "state_name"])
medianaPriceUSDPerM2 = gruposMerge.groupby(by=["property_type", "Indice"])[['price_usd_per_m2']].median()
medianaPriceUSDPerM2.reset_index(inplace=True)
medianaPriceUSDPerM2.set_index(["Indice"], inplace=True)
data_clean.loc[gruposMerge.Indice, "price_usd_per_m2"] = medianaPriceUSDPerM2.price_usd_per_m2
print("Nulos después: ", data_clean.price_usd_per_m2.isnull().sum())

data_clean.reset_index(inplace=True)

Nulos antes:  20365
Nulos después:  20283


In [21]:
dfOtros1 = data_clean[(data_clean.price_aprox_usd.notnull()) & (data_clean.surface_total_in_m2.notnull()) & (data_clean.price_usd_per_m2.isnull())]

In [22]:
dfOtros2 = data_clean[(data_clean.price_aprox_usd.notnull()) & (data_clean.surface_covered_in_m2.notnull()) & (data_clean.price_usd_per_m2.isnull())]

In [23]:
data_clean.loc[dfOtros2.index, "price_usd_per_m2"] = data_clean.loc[dfOtros2.index, "price_aprox_usd"] / data_clean.loc[dfOtros2.index, "surface_covered_in_m2"]

<span style="font-size:18px">IMPUTAR VALORES EN surface_total_in_m2 CON price_aprox_usd / price_usd_per_m2<span>

In [24]:
surfaceTotalInM2NullMask = (data_clean.surface_total_in_m2.isnull()) & \
                           (data_clean.price_aprox_usd.notnull()) & \
                           (data_clean.price_usd_per_m2.notnull())

print("Antes: ", data_clean.surface_total_in_m2.isnull().sum())
data_clean.loc[surfaceTotalInM2NullMask, "surface_total_in_m2"] = \
    data_clean.loc[surfaceTotalInM2NullMask, "price_aprox_usd"] / data_clean.loc[surfaceTotalInM2NullMask, "price_usd_per_m2"]
print("Despues: ", data_clean.surface_total_in_m2.isnull().sum())

Antes:  37779
Despues:  7094


In [25]:
regex = re.compile("([\d|.]{3,10})\sm2|superficie\s([\d|.]{3,10})", flags = re.IGNORECASE)
regexSuperficie = data_clean[(data_clean.surface_total_in_m2.isnull()) & \
                             ((data_clean.property_type == "PH") | \
                             (data_clean.property_type == "apartment") | \
                             (data_clean.property_type == "house"))].description.apply(lambda x: regex.search(str(x)))
resultadoSuperficie = regexSuperficie.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace("m2", "").replace("superficie", "").strip())
resultadoSuperficieMaskNotNull = resultadoSuperficie.notnull()
resultadoSuperficieSinNulos = resultadoSuperficie[resultadoSuperficieMaskNotNull]
print("Antes: ", data_clean.surface_total_in_m2.isnull().sum())
data_clean.loc[resultadoSuperficieSinNulos.index, "surface_total_in_m2"] = resultadoSuperficieSinNulos.astype(np.float64)
print("Despues: ", data_clean.surface_total_in_m2.isnull().sum())
#data_clean.surface_total_in_m2 = data_clean.surface_total_in_m2.apply(lambda x: str(x).lower().replace("m2", "").strip())
#data_clean.surface_total_in_m2 = data_clean.surface_total_in_m2.astype(np.float64)


Antes:  7094
Despues:  6502


In [26]:
dfSuperficies1 = data_clean.loc[resultadoSuperficieSinNulos.index, :]
dfSuperficies2 = dfSuperficies1[(dfSuperficies1.price_aprox_usd.notnull()) & (dfSuperficies1.price_usd_per_m2.isnull())]
dfSuperficies2.filter(items=["price_aprox_usd", "surface_total_in_m2"])

,price_aprox_usd,surface_total_in_m2


In [29]:
data_clean.drop(
    ['Unnamed: 0', 'Indice'], \
    axis=1, inplace=True)

In [33]:
data_clean.to_csv("CleanCSV/Clean03_PreciosYSuperficie.csv")